# Baseline MLP Model
## Training a Feed-foward Neural Network with node features

In [ ]:
from utils import get_data
import os
os.chdir("../")

import pandas as pd
import numpy as np

!bash setup.sh

## Read data and upload to S3

In [ ]:
import io
from scipy.sparse import csr_matrix, save_npz  

train_X, train_y, test_X, test_y = get_data()

train_X.loc[:, 0] = train_y.values
sparse_matrix = csr_matrix(train_X.values)
filename = 'mlp-fraud-dataset.npz'
save_npz(filename, sparse_matrix, compressed=True)

In [ ]:
import os
import sagemaker
from sagemaker.s3 import S3Uploader

from sagemaker_graph_fraud_detection import config

role = config.role

session = sagemaker.Session()
bucket = config.solution_bucket
prefix = 'mlp-fraud-detection'

s3_train_data = S3Uploader.upload(filename, 's3://{}/{}/{}'.format(bucket, prefix,'train'))
print('Uploaded training data location: {}'.format(s3_train_data))

output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('Training artifacts will be uploaded to: {}'.format(output_location))

## Train SageMaker MXNet Estimator

In [ ]:
from sagemaker import get_execution_role
from sagemaker.mxnet import MXNet

params = {'num-gpus': 1,
          'n-layers': 5,
          'n-epochs': 100,
          'optimizer': 'adam',
          'lr': 1e-2
        } 

mlp = MXNet(entry_point='baselines/mlp_fraud_entry_point.py',
            role=role, 
            train_instance_count=1, 
            train_instance_type='ml.p3.2xlarge',
            framework_version="1.4.1",
            py_version='py3',
            hyperparameters=params,
            output_path=output_location,
            code_location=output_location,
            sagemaker_session=session)

mlp.fit({'train': s3_train_data})

In [ ]:
from sagemaker.predictor import json_serializer

predictor = mlp.deploy(initial_instance_count=1,
                       endpoint_name="mlp-fraud-endpoint",
                       instance_type='ml.p3.2xlarge')

# Specify input and output formats.
predictor.content_type = 'text/json'
predictor.serializer = json_serializer
predictor.deserializer = None

In [ ]:
import json

def predict(current_predictor, data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = []
    for array in split_array:
        predictions.append(np.array(json.loads(current_predictor.predict(array.tolist()))))
    return np.concatenate(tuple(predictions), axis=0)

raw_preds = predict(predictor, test_X.values[:, 1:])
y_preds = np.where(raw_preds > 0.5, 1, 0)

In [ ]:
from sklearn.metrics import confusion_matrix, roc_curve, auc
from matplotlib import pyplot as plt
%matplotlib inline

def print_metrics(y_true, y_predicted):

    cm  = confusion_matrix(y_true, y_predicted)
    true_neg, false_pos, false_neg, true_pos = cm.ravel()
    cm = pd.DataFrame(np.array([[true_pos, false_pos], [false_neg, true_neg]]),
                                    columns=["labels positive", "labels negative"],
                                    index=["predicted positive", "predicted negative"])
    
    acc = (true_pos + true_neg)/(true_pos + true_neg + false_pos + false_neg)
    precision = true_pos/(true_pos + false_pos) if (true_pos + false_pos) > 0 else 0
    recall = true_pos/(true_pos + false_neg) if (true_pos + false_neg) > 0 else 0
    f1 = 2*(precision*recall)/(precision + recall) if (precision + recall) > 0 else 0
    print("Confusion Matrix:")
    print(pd.DataFrame(cm, columns=["labels positive", "labels negative"], 
                       index=["predicted positive", "predicted negative"]))
    print("f1: {:.4f}, precision: {:.4f}, recall: {:.4f}, acc: {:.4f}".format(f1, precision, recall, acc))
    print()
    
def plot_roc_curve(fpr, tpr, roc_auc):
    f = plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Model ROC curve')
    plt.legend(loc="lower right")

print_metrics(test_y, y_preds)
fpr, tpr, _ = roc_curve(test_y, y_preds)
roc_auc = auc(fpr, tpr)
plot_roc_curve(fpr, tpr, roc_auc)

In [ ]:
predictor.delete_endpoint()